In [1]:
from typing import List, Tuple
from helper.helper_functions import load_dataset, save_model, get_features_and_target, encode_all_features
from helper.clfmodel_functions import tune_model, seq_feat_selection, multi_metric_cv, plot_multi_score_cv_results, forward_feat_selection_hypertuning
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier

from helper.fairness_functions import split_male_female_metrics, statistical_measures, print_statistical_measures, get_male_female_data, get_male_female_test_data

pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[Reductions]'
pip install 'aif360[Reductions]'
pip install 'aif360[inFairness]'
pip install 'aif360[Reductions]'
pip install 'aif360[OptimalTransport]'


### Loading the cleaned dataset

In [2]:
data = load_dataset('../data/assignment2_income_cleaned.xlsx')

### Feature Engineering (encoding) & Train-Test Split

In [3]:
# Splitting the data into features (X) and target (y)
X, y = get_features_and_target(data, 'income')
# Encoding the features and target, and excluding some columns
X_encoded, y_encoded = encode_all_features(X, y, [])
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y_encoded, test_size=0.2, random_state=42)

### Model

Here, we quickly train and evaluate a Random Forest model with random parameters for demonstration.

In [4]:
# Random Forest Classifier model
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)
dt_preds = rf_model.predict(X_test)
dt_accuracy = accuracy_score(y_test, dt_preds)

print(classification_report(y_test, dt_preds))
print("Decision Tree Accuracy:", dt_accuracy)

              precision    recall  f1-score   support

           0       0.81      0.83      0.82      1175
           1       0.66      0.63      0.65       625

    accuracy                           0.76      1800
   macro avg       0.74      0.73      0.73      1800
weighted avg       0.76      0.76      0.76      1800

Decision Tree Accuracy: 0.7611111111111111


### Feature Importance using the model itself

In [5]:
# Extract feature importances
feature_importances = rf_model.feature_importances_
feature_importance_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': feature_importances
})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

In [6]:
feature_importance_df

,Feature,Importance
0,age,0.306186
1,education,0.173713
2,workinghours,0.154453
9,marital status_Husband,0.066525
22,occupation_Management/Business,0.026475
10,marital status_Never married,0.025336
29,"occupation_Science, Engineering, Technology",0.021523
30,occupation_Service/Hospitality,0.019419
34,workclass_private,0.014913
5,sex_Male,0.014671


### Hyperparameter tuning

Here, we define the base parameter grid for our hyperparameter tuning function.

In [7]:
param_grid = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False],
    'criterion': ['gini', 'entropy', 'log_loss'],
    'random_state': [42]
}

Below, we tune the hyperparameters of the Random Forest model using the defined parameter grid and using all features.

In [ ]:
best_params, best_model, best_accuracy = tune_model(RandomForestClassifier(), X_train, y_train, X_test, y_test, param_grid)

print("\nBest Hyperparameters:", best_params)
print("Best Model:", best_model)
print("Best Model Accuracy:", best_accuracy)

Fitting 5 folds for each of 1350 candidates, totalling 6750 fits


Below, we tune the hyperparameters of the Random Forest model using the defined parameter grid and using a **subset of features**. We exclude the columns 'age', 'ability to speak english', and 'workclass'.

In [ ]:
if False:
    # Splitting the data into features (X) and target (y)
    X_, y_ = get_features_and_target(data, 'income')
    columns_to_exclude = ['age', 'ability to speak english', 'workclass']
    X_ = X_.drop(columns=columns_to_exclude)
    # Encoding the features and target, and excluding some columns
    X_encoded_, y_encoded_ = encode_all_features(X_, y_, columns_to_exclude)
    X_train_, X_test_, y_train_, y_test_ = train_test_split(X_encoded_, y_encoded_, test_size=0.2, random_state=42)
    
    best_params, best_model, best_accuracy = tune_model(RandomForestClassifier(), X_train_, y_train_, X_test_, y_test_, param_grid)
    
    print("\nBest Hyperparameters:", best_params)
    print("Best Model:", best_model)
    print("Best Model Accuracy:", best_accuracy)

In [ ]:
# Forward feature selection with hyperparameter tuning
if False:
    best_subset, best_params, best_score = forward_feat_selection_hypertuning(RandomForestClassifier(), param_grid, X_train, y_train, X_test, y_test)
    
    print("Best subset of features:", best_subset)
    print("Best hyperparameters:", best_params)
    print("Best model accuracy:", best_score)
    
    # Use the best subset and best hyperparameters for final model
    final_model = RandomForestClassifier(**best_params)
    final_model.fit(X_train[best_subset], y_train)
    final_model_preds = final_model.predict(X_test[best_subset])
    final_model_accuracy = accuracy_score(y_test, final_model_preds)
    
    print(classification_report(y_test, final_model_preds))

In [ ]:
# Define the parameter grid for AdaBoost
param_grid = {
    'algorithm': ['SAMME', 'SAMME.R'],
    'n_estimators': [50, 75, 100, 150],
    'learning_rate': [0.1, 0.5, 1.0],
    'estimator': [RandomForestClassifier()],
    'random_state': [42]
}

In [ ]:
if False:
    best_params, best_model, best_accuracy = tune_model(AdaBoostClassifier(), X_train, y_train, X_test, y_test, param_grid)
    
    print("\nBest Hyperparameters:", best_params)
    print("Best Model:", best_model)
    print("Best Model Accuracy:", best_accuracy)

In [ ]:
# Splitting the data into features (X) and target (y)
X_, y_ = get_features_and_target(data, 'income')
X_male, X_female = get_male_female_data(X_, False)
columns_to_exclude = ['sex', 'gave birth this year', 'marital status']
X__ = X_.drop(columns=columns_to_exclude)
# Encoding the features and target, and excluding some columns
X_encoded_, y_encoded_ = encode_all_features(X__, y_, columns_to_exclude)
X_train_, X_test_, y_train_, y_test_ = train_test_split(X_encoded_, y_encoded_, test_size=0.2, random_state=42)

X_male_test, y_male_test, X_female_test, y_female_test = get_male_female_test_data(X_male, X_female, X_test_, y_test_)

best_params, best_model, best_accuracy = tune_model(AdaBoostClassifier(), X_train_, y_train_, X_test_, y_test_, param_grid)

print("\nBest Hyperparameters:", best_params)
print("Best Model:", best_model)
print("Best Model Accuracy:", best_accuracy)

In [ ]:
split_testsets = [X_male_test, y_male_test, X_female_test, y_female_test]
model = AdaBoostClassifier(**best_params)
model.fit(X_train_, y_train_)
fpr_male, fpr_female, tpr_male, tpr_female = split_male_female_metrics(model, X_test, y_test, split_testsets=split_testsets)

print("Male FPR:", fpr_male)
print("Male TPR:", tpr_male)
print("Female FPR:", fpr_female)
print("Female TPR:", tpr_female)

In [ ]:
y_pred = model.predict(X_test_)
X_test_with_sex = X_test_.join(X_['sex'])
X_test_with_sex['sex'] = X_test_with_sex['sex'].map({'Female': 0, 'Male': 1})
DI, DS, EO, EOdds, conf_matrix = statistical_measures(X_test_with_sex, y_test_, y_pred, 'sex', use_lib_implementation=False)
print_statistical_measures(DI, DS, EO, EOdds)

In [ ]:
# save model
save_model(rf_model, '../output/saved_models/random_forest_model.joblib')